## Imagina se eu pudesse ter uma pontuação de cada aluno por intervalo de avaliação. Essa pontuação seria prevista por regressão ajuste de parametros ou por previsão de uma nota na prova baseado naqueles atributos que eu ja coletei

In [1]:
import re
import os
import math
import pandas as pd
import datetime

In [2]:
path = 'Dataset/2018-1/'

In [3]:
dictHorarios = {}

In [4]:
#Coleta a data inicial e final de cada exercicio junto com o seu horario
def coletaDatas(arquivo):
    horarios = []
    hBegin = -1
    hEnd = -1
    dataIni = -1
    dataFim = -1
    horaIni = -1
    horaFim = -1
    for linha in arquivo:
        busca = re.findall(r'\d{4}-\d{1,2}-\d{1,2} \d+:\d+:\d+\.\d+',linha)
        if(len(busca) > 0):
            horaIni = busca[0]
            horarios.append(horaIni)
    if(len(horarios)>0):
        hBegin = horarios[0]
        hEnd = horarios[-1]
    if(hBegin != -1):
        busquinha = re.findall(r'\d{4}-\d{1,2}-\d{1,2}',hBegin)
        if(len(busquinha) > 0):
            dataIni = busquinha[0]
        busquinha = re.findall(r'\d+:\d+:\d+\.\d+',hBegin)
        if(len(busquinha) > 0):
            horaIni = busquinha[0]
    if(hEnd != -1):
            busquinha = re.findall(r'\d{4}-\d{1,2}-\d{1,2}',hEnd)
            if(len(busquinha)>0):
                    dataFim = busquinha[0]
            busquinha = re.findall(r'\d+:\d+:\d+\.\d+',hEnd)
            if(len(busquinha) > 0):
                horaFim = busquinha[0]
    '''            
    print("data raw:")
    print(hBegin,hEnd)
    print(dataIni, dataFim)
    print("Datas")
    print(horaIni,horaFim)
    print("EOF")
    '''
    return horarios, dataIni, dataFim, horaIni, horaFim

In [5]:
#Calcula a quantidade de linhas de um arquivo de log
def qntd_linhas_log(arquivo):
    contador = 0
    paste = 0
    delete = 0
    for linha in arquivo.split("\n"):
        contador += 1
        busca = re.search(r'paste',linha)
        busca2 = re.search(r'delete',linha)
        busca3 = re.search(r'Backspace',linha)
        if(busca): paste += 1
        if(busca2): delete += 1
        if(busca3): delete += 1
    
    p1 = paste/contador
    p2 = delete/contador
    return contador,round(p1,2),round(p2,2)
    

In [6]:
#Calcula o tempo empenhado no exercicio
def tempoEmpenhado(data_ini, data_fim, hora_ini, hora_fim):
    s = '' + str(data_ini) + ' ' + str(hora_ini) 
    r = '' + str(data_fim) + ' ' +str(hora_fim)
    date1 = int(datetime.datetime.strptime(s, '%Y-%m-%d %H:%M:%S.%f').strftime("%s"))
    date2 = int(datetime.datetime.strptime(r, '%Y-%m-%d %H:%M:%S.%f').strftime("%s"))
    difdate = date2 - date1
    return difdate

In [7]:
#Calcula a quantidade de dias em que o mesmo exercicio foi resolvido
def n_acessosColetor(arquivo):
    primeiroAcesso = 1
    difdate = 0
    for linha in arquivo:
        busca = re.search(r'\d{4}-\d{1,2}-\d{1,2}',linha)
        if(busca):
            if(primeiroAcesso):
                dataIni = busca.group(0)
                primeiroAcesso = 0
            dataAtual = busca.group(0)
            #date1 = int(datetime.datetime.strptime(dataIni, '%Y-%m-%d').strftime("%d"))
            #date2 = int(datetime.datetime.strptime(dataAtual, '%Y-%m-%d').strftime("%d"))
            #difdate = date2 - date1
            start_date = datetime.datetime.strptime(dataIni, '%Y-%m-%d')
            end_date = datetime.datetime.strptime(dataAtual, '%Y-%m-%d')
            difdate = abs(end_date-start_date)
            difdate = difdate.days
            

            
    
    return difdate

In [8]:
#Ve a quantidade de submissoes em um exercicio
def attemptsColetor(arquivo):
    contador = 0
    for linha in arquivo.split("\n"):
        busca = re.search(r'SUBMITION',linha)
        if(busca): contador += 1
    return contador

In [9]:
#Ve a quantidade de testes em um exercicio
def testColetor(arquivo):
    contador = 0
    for linha in arquivo.split("\n"):
        busca = re.search(r'TEST',linha)
        if(busca): contador += 1
    return contador

In [10]:
#Ve a quantidade de exercicios em um laboratorio
def exercicios_no_lab(arquivo):
    qntd = 0
    for linha in arquivo.split("\n"):
        busca = re.search(r'number of exercises: \d+',linha)
        if(busca): 
            busca2 = re.search(r'\d+',busca.group(0))
            return busca2.group(0)

In [11]:
#Ve a quantidade de exercicios respondidos corretamente em um laboratorio
def exer_certos_no_lab(arquivo):
    qntd = 0
    for linha in arquivo.split("\n"):
        busca = re.search(r'correct: \d+',linha)
        if(busca): 
            busca2 = re.search(r'\d+',busca.group(0))
            return busca2.group(0)

In [12]:
#Nota final por laboratorio
def nota_lab(arquivo):
    qntd = 0
    for linha in arquivo.split("\n"):
        busca = re.search(r'grade \(0-10\): \d+\.\d+',linha)
        if(busca):
            busca2 = re.search(r'\d+\.\d+',busca.group(0))
            return busca2.group(0)

In [13]:
#Quantidade de linhas de um codigo
def qntd_linhas(arquivo):
    qntd = 0
    for linha in arquivo.split("\n"):
        busca = re.search(r'\w*',linha)
        if(busca): qntd += 1
    return qntd

In [14]:
#Quantidade de linhas de um codigo sem comentarios etc
def qntd_linhas_filtrado(arquivo):
    qntd = 0
    for linha in arquivo.split("\n"):
        busca = re.search(r'\S',linha)
        if(busca): 
            busca2 = re.search(r'^(?!#).*',busca.group(0))
            if(busca2): qntd += 1

    return qntd

In [15]:
def descobre_atividade(arquivo,exercicio):
    for linha in arquivo.split("\n"):
        busca = re.search(r'type: exam',linha)
        if(busca): 
            busca2 = re.search(r'\d+',exercicio)
            return busca2.group(0)

In [16]:
def coleta_nota_grade(arquivo):
     for linha in arquivo.split("\n"):
        nota = re.search(r'\: \d+.*\d*',linha)
        if(nota):
            notaLimpa = re.search(r'\d+.*\d*',nota.group(0))
            return notaLimpa.group(0)

In [17]:
def coleta_corretos_grade(arquivo):
    for linha in arquivo.split("\n"):
        corretos = re.search(r'\bcorrect\b: \d+',linha)
        if(corretos):
            corretos = re.search(r'\d+',corretos.group(0))
            return corretos.group(0)

In [18]:
def coleta_incorretos_grade(arquivo):
    for linha in arquivo.split("\n"):
        incorretos = re.search(r'\bincorrect\b: \d+',linha)
        if(incorretos):
            incorretos = re.search(r'\d+',incorretos.group(0))
            return incorretos.group(0)

In [19]:
def coleta_brancos_grade(arquivo):
    for linha in arquivo.split("\n"):
        brancos = re.search(r'blank: \d+',linha)
        if(brancos):
            brancos = re.search(r'\d+',brancos.group(0))
            return brancos.group(0)

In [20]:
def coletor_grade(arquivo):
    for linha in arquivo.split("\n"):
        nota = re.search(r'\: \d+.\d+',linha)
        corretos = re.search(r'\bcorrect\b: \d+',linha)
        incorretos = re.search(r'\bincorrect\b: \d+',linha)
        brancos = re.search(r'blank: \d+',linha)
        if(nota):
            notaLimpa = re.search(r'\d+.\d+',nota.group(0))
            notaF = notaLimpa.group(0)
        if(corretos):
            corretos = re.search(r'\d+',corretos.group(0))
            corretosF = corretos.group(0)
        if(incorretos):
            incorretos = re.search(r'\d+',incorretos.group(0))
            incorretosF = incorretos.group(0)
        if(brancos):
            brancos = re.search(r'\d+',brancos.group(0))
            brancosF = brancos.group(0)
    print(notaF,corretosF,incorretosF,brancosF)
   
        

In [21]:
#Definindo o DataFrame
colunas = ['Aluno','Lab','Exercicio','Duracao','Qntd_Linhas','Qntd_Linhas_Filtrado','Testes','Submissoes','N_Dias','PP_Paste','PP_Delete','Linhas_Log','Nota']
df = pd.DataFrame(columns = colunas)
print(df)
df = df[0:0]

Empty DataFrame
Columns: [Aluno, Lab, Exercicio, Duracao, Qntd_Linhas, Qntd_Linhas_Filtrado, Testes, Submissoes, N_Dias, PP_Paste, PP_Delete, Linhas_Log, Nota]
Index: []


In [22]:
colunas = ['Aluno','Lab','Corretos','Incorretos','Branco','Nota']
dfGrade = pd.DataFrame(columns= colunas)

In [23]:
for folder in os.listdir(path):
    if not folder.startswith('.'):
        
        directory = os.path.join(path,folder)+'/'
        turma = folder
        PastaAtividades = directory + 'assessments'
        pastaAlunos = directory + 'users'
        for file in os.listdir(PastaAtividades):
            vetorProvas = []
            numProva = file
            file = os.path.join(PastaAtividades,file)
            arquivo = open(file,'r',encoding='ISO-8859-1')
            data = arquivo.read()
            prova = descobre_atividade(data,numProva)
            if(prova != None): vetorProvas.append(prova)
        
        for alunoUnidade in os.listdir(pastaAlunos):
            aluno = alunoUnidade
            pastaAluno = pastaAlunos + '/' + aluno
            pastaNotas = pastaAluno + '/grades'
            
            for prova in os.listdir(pastaNotas):
                file = os.path.join(pastaNotas,prova)
                try:
                    arquivo = open(file,'r',encoding='ISO-8859-1')
                    data = arquivo.read()
                    brancos = coleta_brancos_grade(data)
                    corretos = coleta_corretos_grade(data)
                    incorretos = coleta_incorretos_grade(data)
                    nota = coleta_nota_grade(data)
                    if(nota != None):
                        labson = re.search(r'\d+',prova)
                        labson = labson.group(0)
                        dfGrade = dfGrade.append({'Aluno': aluno,'Lab' : labson,'Corretos':corretos,'Incorretos':incorretos,'Branco': brancos,'Nota':nota}, ignore_index=True)
                except FileNotFoundError:
                    print(file," nao existe")
        

In [24]:
dfGrade['Aluno'] = dfGrade['Aluno'].astype(int)
dfGrade['Lab'] = dfGrade['Lab'].astype(int)
dfGrade['Corretos'] = dfGrade['Corretos'].astype(int)
dfGrade['Incorretos'] = dfGrade['Incorretos'].astype(int)
dfGrade['Branco'] = dfGrade['Branco'].astype(int)
dfGrade['Nota'] = dfGrade['Nota'].astype(float) 

In [25]:
df_csv = pd.read_csv('dadosPorLabMean.csv')

In [26]:
df_csv = df_csv.drop(['Nota'],axis = 1)

In [27]:
df_csv.columns

Index(['Aluno', 'Lab', 'PP_Delete', 'Duracao', 'PP_Paste', 'Qntd_Linhas',
       'Linhas_Log', 'Qntd_Linhas_Filtrado', 'Submissoes', 'Testes'],
      dtype='object')

In [28]:
dfGrade.columns

Index(['Aluno', 'Lab', 'Corretos', 'Incorretos', 'Branco', 'Nota'], dtype='object')

In [29]:
df

,Aluno,Lab,Exercicio,Duracao,Qntd_Linhas,Qntd_Linhas_Filtrado,Testes,Submissoes,N_Dias,PP_Paste,PP_Delete,Linhas_Log,Nota


In [30]:
df_unido = (pd.merge(df_csv,dfGrade, on=['Aluno','Lab']))

In [31]:
df_unido

,Aluno,Lab,PP_Delete,Duracao,PP_Paste,Qntd_Linhas,Linhas_Log,Qntd_Linhas_Filtrado,Submissoes,Testes,Corretos,Incorretos,Branco,Nota
0,1085,1031,0.530000,8.000000e+01,0.000000,25.000000,19.000000,9.000000,0.000000,1.000000,0,0,12,0.000000
1,1085,1083,0.170000,1.480000e+02,0.005000,12.000000,186.000000,5.500000,2.000000,6.000000,2,0,8,2.000000
2,1085,1118,0.172500,9.562500e+02,0.002500,28.000000,871.000000,20.000000,5.750000,20.500000,3,1,8,2.500000
3,1085,1265,0.300000,1.864000e+03,0.000000,32.000000,991.000000,20.000000,17.000000,42.000000,1,0,11,0.833333
4,1163,1012,0.216429,5.514350e+04,0.007143,2.500000,82.785714,2.071429,2.428571,1.214286,14,0,0,10.000000
5,1163,1029,0.211667,4.900946e+05,0.003333,10.666667,416.333333,6.166667,2.833333,6.416667,12,0,0,10.000000
6,1163,1051,0.227500,5.140167e+04,0.004167,9.083333,460.750000,6.833333,2.666667,13.166667,12,0,0,10.000000
7,1163,1052,0.156667,3.630000e+02,0.003333,8.333333,300.000000,5.666667,1.000000,3.000000,3,0,0,10.000000
8,1163,1084,0.204000,1.665611e+05,0.012000,14.500000,415.800000,8.600000,6.900000,13.300000,10,0,0,10.000000
9,1163,1089,0.023333,6.829733e+04,0.091667,12.166667,13.333333,8.333333,1.333333,3.333333,6,0,0,10.000000


In [34]:
df_unido[df_unido['Nota']>5.0]

,Aluno,Lab,PP_Delete,Duracao,PP_Paste,Qntd_Linhas,Linhas_Log,Qntd_Linhas_Filtrado,Submissoes,Testes,Corretos,Incorretos,Branco,Nota
4,1163,1012,0.216429,5.514350e+04,0.007143,2.500000,82.785714,2.071429,2.428571,1.214286,14,0,0,10.00000
5,1163,1029,0.211667,4.900946e+05,0.003333,10.666667,416.333333,6.166667,2.833333,6.416667,12,0,0,10.00000
6,1163,1051,0.227500,5.140167e+04,0.004167,9.083333,460.750000,6.833333,2.666667,13.166667,12,0,0,10.00000
7,1163,1052,0.156667,3.630000e+02,0.003333,8.333333,300.000000,5.666667,1.000000,3.000000,3,0,0,10.00000
8,1163,1084,0.204000,1.665611e+05,0.012000,14.500000,415.800000,8.600000,6.900000,13.300000,10,0,0,10.00000
9,1163,1089,0.023333,6.829733e+04,0.091667,12.166667,13.333333,8.333333,1.333333,3.333333,6,0,0,10.00000
10,1163,1106,0.156667,6.296667e+02,0.006667,11.666667,493.666667,9.666667,1.000000,8.666667,3,0,0,10.00000
11,1163,1129,0.148333,8.319317e+04,0.033333,25.583333,797.833333,21.916667,10.833333,24.000000,12,0,0,10.00000
13,1163,1152,0.180000,8.856667e+02,0.003333,23.666667,891.333333,20.333333,1.666667,10.333333,3,0,0,10.00000
14,1163,1155,0.184167,1.285743e+05,0.015833,14.583333,481.083333,9.666667,2.750000,5.833333,12,0,0,10.00000


In [35]:
df_unido.to_csv('dadosProvaOnly.csv', encoding='utf-8', index=False)